In [41]:
import tkinter as tk
from tkinter import filedialog, simpledialog, messagebox
import os
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import defaultdict
from PIL import Image, ImageTk
from PyPDF2 import PdfReader

In [14]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\adity\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\adity\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [63]:
class ResumeFilterApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Resume Filter Application")
        self.root.geometry("600x400")
        self.root.config(bg="#f0f0f0")
        
        self.logo_path = "C:/Users/adity/OneDrive/Desktop/image.png" 
        self.load_image(self.logo_path, 50, 50)

        self.job_role_label = tk.Label(root, text="Enter Job Role:", font=("Helvetica", 14), bg="#f0f0f0")
        self.job_role_label.pack()
        self.job_role_entry = tk.Entry(root, width=50, font=("Helvetica", 12))
        self.job_role_entry.pack()
        
        self.job_skills_label = tk.Label(root, text="Enter Job Skills (comma separated):", font=("Helvetica", 14), bg="#f0f0f0")
        self.job_skills_label.pack()
        self.job_skills_entry = tk.Entry(root, width=50, font=("Helvetica", 12))
        self.job_skills_entry.pack()
        
        self.add_resume_button = tk.Button(root, text="Add Resume", command=self.add_resume, font=("Helvetica", 12))
        self.add_resume_button.pack(pady=10)
        self.add_resume_button.bind("<Enter>", self.on_enter)
        self.add_resume_button.bind("<Leave>", self.on_leave)
        
        self.filter_resumes_button = tk.Button(root, text="Filter Resumes", command=self.filter_resumes, font=("Helvetica", 12))
        self.filter_resumes_button.pack(pady=10)
        self.filter_resumes_button.bind("<Enter>", self.on_enter)
        self.filter_resumes_button.bind("<Leave>", self.on_leave)
        
        self.resume_texts = []
        self.resume_names = []
        self.resume_contacts = []
        
    def load_image(self, path, width, height):
        image = Image.open(path)
        image = image.resize((width, height))
        self.logo_image = ImageTk.PhotoImage(image)
        self.logo_label = tk.Label(self.root, image=self.logo_image, bg="#f0f0f0")
        self.logo_label.image = self.logo_image
        self.logo_label.pack()

    def add_resume(self):
        file_path = filedialog.askopenfilename(filetypes=[("PDF files", "*.pdf")])
        if file_path:
            content = self.extract_text_from_pdf(file_path)
            if content:
                name = simpledialog.askstring("Input", "Enter the candidate's name:")
                contact = simpledialog.askstring("Input", "Enter the candidate's contact details:")
                self.resume_texts.append(content)
                self.resume_names.append(name)
                self.resume_contacts.append(contact)
                messagebox.showinfo("Info", f"Resume for {name} added successfully!")
            else:
                messagebox.showerror("Error", "Failed to extract text from the PDF file.")
    
    def extract_text_from_pdf(self, file_path):
        try:
            reader = PdfReader(file_path)
            content = ""
            for page in reader.pages:
                content += page.extract_text()
            return content
        except Exception as e:
            print(f"Error extracting text from PDF: {e}")
            return None
    
    def filter_resumes(self):
        job_role = self.job_role_entry.get()
        job_skills = self.job_skills_entry.get().split(',')
        job_skills = [skill.strip().lower() for skill in job_skills]

        if not job_role or not job_skills:
            messagebox.showerror("Error", "Please enter both job role and skills.")
            return

        stop_words = set(stopwords.words('english'))
        total_possible_score = len(job_skills)  # Calculate total possible score
        filtered_scores = defaultdict(int)

        for i, resume in enumerate(self.resume_texts):
            tokens = word_tokenize(resume.lower())
            filtered_tokens = [word for word in tokens if word.isalnum() and word not in stop_words]

            score = 0
            for skill in job_skills:
                score += filtered_tokens.count(skill)

            # Adjust score to prevent exceeding 100%
            score = min(score, total_possible_score)

            filtered_scores[i] = (score / total_possible_score) * 100  # Calculate and store percentage

        sorted_resumes = sorted(filtered_scores.items(), key=lambda item: item[1], reverse=True)

        result_text = "Name".ljust(20) + "Contact".ljust(20) + "Match Percentage\n"
        for index, score in sorted_resumes:
            if score > 0:
                result_text += f"{self.resume_names[index]}".ljust(20) + f"{self.resume_contacts[index]}".ljust(20) + f"{score:.2f}%\n"  # Display percentage

        result_window = tk.Toplevel(self.root)
        result_window.title("Filtered Resumes")
        result_label = tk.Label(result_window, text=result_text, font=("Helvetica", 12), bg="#f0f0f0")
        result_label.pack()

    def on_enter(self, event):
        event.widget['background'] = '#b3b3b3'
        
    def on_leave(self, event):
        event.widget['background'] = 'SystemButtonFace'


In [64]:
if __name__ == "__main__":
    root = tk.Tk()
    app = ResumeFilterApp(root)
    root.mainloop()